### Verificar se a biblioteca da OCI está baixada

In [36]:
!oci --version

3.54.2


##### Caso não esteja, rodar o comando:

!pip install oci

### Configure um arquivo com credenciais API Key
- No terminal rode o comado: 
> oci setup config
- Adicionar OCID (no seu perfil > user_setings)
- Adicionar Tenanty OCID (no seu perfil > tenancy)
- Deixar pastas padrão
- Configurar chave pública(perfil > user_settings > tokens and keys > add api key > paste a public key)

### Código direto do code examples no OCI GenAI
- Llama 3.3 70B 

⚠️⚠️⚠️⚠️⚠️ O código não funciona se você não alterar para o seu compartment_id e conferir model_id ⚠️⚠️⚠️⚠️⚠️⚠️

In [10]:
user_input = "O que é uma llama?"

In [11]:
import oci
import json

# Setup basic variables
# Auth Config
# TODO: Please update config profile name and use the compartmentId that has policies grant permissions for using Generative AI Service
compartment_id = "ocid1.tenancy.oc1..aaaaaaaakzuzfystmhyj7je5x7ymwd3ofd7wb6rtsfrx7nvbo272vta3rwna"
CONFIG_PROFILE = "SAOPAULO"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# Service endpoint
endpoint = "https://inference.generativeai.sa-saopaulo-1.oci.oraclecloud.com"

generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint)
chat_detail = oci.generative_ai_inference.models.ChatDetails()

content = oci.generative_ai_inference.models.TextContent()
content.text = f"{user_input}"
message = oci.generative_ai_inference.models.Message()
message.role = "USER"
message.content = [content]

chat_request = oci.generative_ai_inference.models.GenericChatRequest()
chat_request.api_format = oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC
chat_request.messages = [message]
chat_request.max_tokens = 600
chat_request.temperature = 1
chat_request.frequency_penalty = 0
chat_request.presence_penalty = 0
chat_request.top_p = 0.75

chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.sa-saopaulo-1.amaaaaaask7dceyarsn4m6k3aqvvgatida3omyprlcs3alrwcuusblru4jaa")
chat_detail.chat_request = chat_request
chat_detail.compartment_id = compartment_id

chat_response = generative_ai_inference_client.chat(chat_detail)

# Print result
print("**************************Chat Result**************************")
choices = chat_response.data.chat_response.choices
if choices and len(choices) > 0:
    response_text = choices[0].message.content[0].text
else:
    response_text = "Nenhuma resposta foi gerada pelo modelo."
# Exibindo a resposta no console
print(json.dumps({"response": response_text}, indent=4, ensure_ascii=False))

**************************Chat Result**************************
{
    "response": "Uma llama é um mamífero da família dos camelídeos, nativo da América do Sul, principalmente nos Andes. É um animal herbívoro, conhecido por sua pelagem macia e sua habilidade de carregar cargas. As lhamas são frequentemente utilizadas como animais de carga e também são valorizadas por sua lã. Elas são inteligentes e sociais, vivendo em rebanhos no seu habitat natural."
}


### Multimodal
- Llama 3.2 90B - processando imagens

In [12]:
# Configuração da mensagem do sistema e do usuário
system_msg = """Você é um especialista em analisar imagens"""
user_msg = "o que tem na imagem? "
image_content = "demo-image.jpeg"

In [ ]:
# coding: utf-8
# Copyright (c) 2023, Oracle and/or its affiliates.  All rights reserved.
# This software is dual-licensed to you under the Universal Permissive License (UPL) 1.0 as shown at https://oss.oracle.com/licenses/upl or Apache License 2.0 as shown at http://www.apache.org/licenses/LICENSE-2.0. You may choose either license.

##########################################################################
# chat_demo.py
# Supports Python 3
##########################################################################
# Info:
# Get texts from LLM model for given prompts using OCI Generative AI Service.
##########################################################################
# Application Command line(no parameter needed)
# python chat_demo.py
##########################################################################
import oci

# Setup basic variables
# Auth Config
# TODO: Please update config profile name and use the compartmentId that has policies grant permissions for using Generative AI Service
compartment_id = "ocid1.compartment.oc1..aaaaaaaa3x7n7wwfnghe4imvt3niwo76wgqv6ecn2iadiwoph73jjowbhbna"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# Service endpoint
endpoint = "https://inference.generativeai.sa-saopaulo-1.oci.oraclecloud.com"

generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))
chat_detail = oci.generative_ai_inference.models.ChatDetails()

###### Código adicionado para suporte a imagem ######
import base64
import json


# Transforma a imagem em base64
try:
    with open(image_content, "rb") as image_file:
        image_data = image_file.read()
    base64_image = base64.b64encode(image_data).decode("utf-8")
    base64_image_url = f"data:image/jpeg;base64,{base64_image}"
except oci.exceptions.ServiceError as e:
    raise ValueError(f"Erro ao acessar a imagem")
    

# Configuração do conteúdo da imagem
imagem = oci.generative_ai_inference.models.ImageContent()
image_url_content = oci.generative_ai_inference.models.ImageUrl()
image_url_content.url = base64_image_url
imagem.image_url = image_url_content

# Configuração do conteúdo de texto
texto = oci.generative_ai_inference.models.TextContent()
texto.text = "O que tem nesta imagem?"

# Configuração da mensagem do usuário
message = oci.generative_ai_inference.models.UserMessage()
message.role = "USER"
message.content = [texto, imagem]
#################################################################

chat_request = oci.generative_ai_inference.models.GenericChatRequest()
chat_request.api_format = oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC
chat_request.messages = [message]
chat_request.max_tokens = 600
chat_request.temperature = 1
chat_request.frequency_penalty = 0
chat_request.presence_penalty = 0
chat_request.top_p = 0.75



chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.sa-saopaulo-1.amaaaaaask7dceyalwceqwzlywqqxfzz3grpzjr42fej5qlybhu2d666oz4q")
chat_detail.chat_request = chat_request
chat_detail.compartment_id = compartment_id

chat_response = generative_ai_inference_client.chat(chat_detail)

# Print result
print("**************************Chat Result**************************")
print(vars(chat_response))


{
    "response": "Claro, posso analisar a imagem e fornecer informações sobre ela. A imagem mostra uma mulher com cabelos pretos e óculos, vestindo uma jaqueta vermelha, ao lado de um animal que parece ser uma lhama. A cena parece ser em um ambiente natural, com árvores e uma cerca de madeira ao fundo. A mulher está sorrindo e olhando para a câmera, enquanto a lhama está olhando para a frente. A imagem parece ser uma selfie, pois a mulher está segurando a câmera com uma das mãos. A qualidade da imagem é boa, com cores vivas e detalhes claros. Não há nada anormal ou suspeito na imagem."
}
